# COVID-19 Cases vs. Tweets

The goal is to analyse how the number of Tweets regarding COVID-19 correlates with the case numbers in Germany. Due to the restrictions of the Twitter API, prominent Twitter accounts of German Politicians will be used.

This analysis could easily be expanded for other countries and their politicians, or a more general population using the geolocation of tweets.

## Ranking of the ten most popular German politicians on social networks in July 2020

<img src="images/german_politicians_statista.png" alt="Drawing" style="width: 600px;"/>

source: https://www.statista.com/statistics/446360/social-media-ranking-of-the-most-popular-politicians-germany/


### Twitter Handles:
N/A <br>
@sebastiankurz <br>
@martinschulz <br>
@SWagenknecht <br>
N/A <br>
@GregorGysi <br>
@JunckerEU <br>
@MartinSonneborn <br>
@c_lindner <br>
@nicosemsrott




### Twitter Hashtags:

## Alternative handles:

@BMG_Bund